In [1]:
import gurobipy as gp
import pandas as pd
import geopandas as gpd
from gurobipy import Model, GRB, quicksum
import numpy as np
import pickle
import os

In [2]:
from typing import Dict, Any

In [3]:
# Códigos IBGE dos 75 municípios de Sergipe
mun_origem_df = pd.read_csv('Downloads/CD_MUN_origem.csv')

# Códigos IBGE de 25 municípios com maior quantidade de atendimentos nas especialidades 
# que foram aprovados pelos gestores de saúde (Qtd. aprovada)
mun_alvo_df = pd.read_csv('Downloads/CD_MUN_alvo.csv')

In [4]:
# Total de Centros Médicos de Especialidades por município [TABNET: CNES - Tipos de Estabelecimento]
centros_df = pd.read_csv('Downloads/cnes_cnv_estabse_115.csv', sep=';')

# Resultado do OPEN ROUTE SERVICE (distâncias entre municípios)
rotas_df = pd.read_csv('Downloads/rotas_75x25_FONTES_DESTINOS.csv')

In [5]:
# Conjuntos
I = mun_origem_df['CD_MUN'].tolist() # Municípios
J = mun_alvo_df['CD_MUN'].tolist() # Municípios candidatos
E = ['Ortopedia','Proctologia','Urologia'] # Especialidades médicas

In [6]:
# Substituir 'caminho/do/seu/arquivo.csv' pelo caminho real

# DEMANDA DOS MUNICÍPIOS SEGUNDO OS PARÂMETROS DO SUS PARA OFERECIMENTO DAS ESPECIALIDADES
# Portaria do SUS
df_demanda = pd.read_csv('Downloads/SE_demandas.csv')

# Define a coluna de municípios como índice
df_demanda.set_index('CD_MUN', inplace=True)

# Renomeia colunas para o formato 'Especialidade' (sem o prefixo 'Demanda_')
# O nome da especialidade deve corresponder aos seus 'sets' no Gurobi (E)
df_demanda.columns = ['Ortopedia', 'Proctologia', 'Urologia']

# Converte para dicionário de dicionários (para HE_ie)
# O formato final será: {'Mun1': {'Ortopedia': 5.2, 'Proctologia': 1.5, ...}, ...}
HE_dict = df_demanda.to_dict(orient='index')

In [7]:
# Para conferir o dicionário HE_ie
print(HE_dict)

{2800100: {'Ortopedia': 0.2202, 'Proctologia': 0.04404, 'Urologia': 0.06605981126}, 2800209: {'Ortopedia': 2.0556, 'Proctologia': 0.41112, 'Urologia': 0.6166782381}, 2800308: {'Ortopedia': 63.0932, 'Proctologia': 12.61864, 'Urologia': 18.92790592}, 2800407: {'Ortopedia': 1.0464, 'Proctologia': 0.20928, 'Urologia': 0.3139191031}, 2800506: {'Ortopedia': 1.8626, 'Proctologia': 0.37252, 'Urologia': 0.5587784035}, 2800605: {'Ortopedia': 4.5175, 'Proctologia': 0.9035, 'Urologia': 1.355246128}, 2800670: {'Ortopedia': 2.5007, 'Proctologia': 0.50014, 'Urologia': 0.7502078565}, 2800704: {'Ortopedia': 0.8025, 'Proctologia': 0.1605, 'Urologia': 0.2407493121}, 2801009: {'Ortopedia': 1.8719, 'Proctologia': 0.37438, 'Urologia': 0.5615683955}, 2801108: {'Ortopedia': 0.385, 'Proctologia': 0.077, 'Urologia': 0.11549967}, 2801207: {'Ortopedia': 3.3641, 'Proctologia': 0.67282, 'Urologia': 1.009227116}, 2801306: {'Ortopedia': 3.2783, 'Proctologia': 0.65566, 'Urologia': 0.98348719}, 2801405: {'Ortopedia': 2

In [8]:
# 1. Parâmetros de Produtividade (Consultas/FTE/ano)
# Calculado com base no Quadro 35 da Portaria 1631: (Consultas/100K) / (Médicos/100K)
PRODUCTIVITY_MAP: Dict[str, float] = {
    'Ortopedia': 1500.0,      # 15.000 / 10 = 1.500
    'Proctologia': 800.0,       # 1.600 / 2.0 = 800
    'Urologia': 1166.67,      # 3.500 / 3.0 ≈ 1166.67
}
    
# Função para calcular a disponibilidade existente KE_ie    
def calculate_ke_ie_dict(file_path: str) -> Dict[int, Dict[str, float]]:
    """
    Carrega os dados de produção, calcula a oferta em FTE (KE_ie) por município e especialidade,
    e retorna o resultado no formato de dicionário de dicionários.

    Args:
        file_path: Caminho para o arquivo CSV contendo a produção de consultas.

    Returns:
        Um dicionário no formato {codigo_mun: {especialidade: KE_ie_FTE}}.
    """
    df = pd.read_csv(file_path)
    
    # Identificar as colunas de código do município e de quantidades
    mun_col = 'CD_MUN'
    qty_cols = [col for col in df.columns if 'Qtd.aprovada' in col]
    
    # Renomear as colunas de quantidade para os nomes das especialidades
    # Ex: 'Ortopedia_Qtd.aprovada' -> 'Ortopedia'
    col_mapping = {col: col.split('_')[0] for col in qty_cols}
    df = df.rename(columns=col_mapping)
    
    # Preparação dos Dados
    # Selecionar apenas o código do município e as colunas de especialidade renomeadas
    specialty_cols = list(col_mapping.values())
    df_prod_wide = df[[mun_col] + specialty_cols].copy()

    # Converter o formato wide (código do município e várias colunas de especialidade)
    # para o formato long (código do município, uma coluna de especialidade, uma coluna de quantidade)
    df_prod_long = df_prod_wide.melt(
        id_vars=mun_col,
        var_name='Especialidade',
        value_name='Producao_Anual_Consultas'
    )
    
    # Cálculo do KE_ie (Oferta Existente em FTE)
    
    def apply_ke_ie_calculation(row: pd.Series) -> float:
        """Aplica a fórmula KE_ie = Produção Anual / Produtividade Anual."""
        especialidade = row['Especialidade']
        producao_anual = row['Producao_Anual_Consultas']
        
        # 3.b Obter a Produtividade
        if especialidade in PRODUCTIVITY_MAP:
            productivity = PRODUCTIVITY_MAP[especialidade]
            
            # 3.c Calcular KE_ie em FTE
            ke_ie = producao_anual / productivity
            return round(ke_ie, 4)  # Arredondar para 4 casas decimais para precisão
        
        return 0.0

    df_prod_long['KE_ie_FTE'] = df_prod_long.apply(apply_ke_ie_calculation, axis=1)

    # Formatar o resultado em dicionário de dicionários
    # Pivotar o DataFrame de volta para o formato wide, com municípios como índice
    ke_ie_df = df_prod_long.pivot(
        index=mun_col, 
        columns='Especialidade', 
        values='KE_ie_FTE'
    )
    
    # Converter o DataFrame pivotado para o dicionário final
    ke_ie_dict = ke_ie_df.to_dict(orient='index')
    
    return ke_ie_dict

# Executar a função com o arquivo
# Produção de consultas nas especialidades, por município
file_path = 'Downloads/SE_mun_producao_consultas.csv'
KE_dict = calculate_ke_ie_dict(file_path)

print("--- Dicionário KE_ie (Oferta Existente em FTE) ---")
print(KE_dict)

--- Dicionário KE_ie (Oferta Existente em FTE) ---
{2800100: {'Ortopedia': 0.0253, 'Proctologia': 0.0088, 'Urologia': 0.0111}, 2800209: {'Ortopedia': 0.2113, 'Proctologia': 0.0537, 'Urologia': 0.1234}, 2800308: {'Ortopedia': 24.902, 'Proctologia': 7.7988, 'Urologia': 7.7545}, 2800407: {'Ortopedia': 0.1447, 'Proctologia': 0.1163, 'Urologia': 0.084}, 2800506: {'Ortopedia': 0.2127, 'Proctologia': 0.0975, 'Urologia': 0.1037}, 2800605: {'Ortopedia': 0.9233, 'Proctologia': 0.1938, 'Urologia': 0.1869}, 2800670: {'Ortopedia': 0.242, 'Proctologia': 0.1338, 'Urologia': 0.1526}, 2800704: {'Ortopedia': 0.0473, 'Proctologia': 0.0037, 'Urologia': 0.0197}, 2801009: {'Ortopedia': 0.1753, 'Proctologia': 0.0775, 'Urologia': 0.2537}, 2801108: {'Ortopedia': 0.0673, 'Proctologia': 0.0013, 'Urologia': 0.0129}, 2801207: {'Ortopedia': 0.2433, 'Proctologia': 0.0688, 'Urologia': 0.1226}, 2801306: {'Ortopedia': 0.2847, 'Proctologia': 0.0875, 'Urologia': 0.1483}, 2801405: {'Ortopedia': 0.17, 'Proctologia': 0.1087

In [9]:
# Definição de Limites Orçamentários e Capacidade (A serem ajustados dinamicamente)
P_max = 5           # Número exato (ou máximo, ver linha abaixo) de NOVOS centros que o governo pode abrir/financiar.
# P_min = 1        # Descomentar aqui e na Restrição 6 caso queira abrir um número de facilidades entre dois limites,
                   # usando o parâmetro acima como máximo. 
Q_max = 100         # Orçamento total de FTEs (Especialistas) NOVOS a serem contratados na rede.
k_min = 1.0     # Mínimo de FTEs por centro aberto
k_max = 5.0    # Máximo de FTEs por centro aberto

# Parâmetro de Trade-off (Alpha no artigo)
# Alpha = 1: Foco total na Eficiência (Acesso Médio). Alpha = 0: Foco total na Equidade (Pior Cenário).
alpha = 0.5

In [10]:
# Parâmetros: Mínimos
F_min = 2           # nº mínimo de centros de saúde

r_km = 1.00     # R$/km (custo por km rodado) [ARTIGO ajustado]
s_kmh = 60.0     # km/h (velocidade média) [ARTIGO]
h = 5.0  # tempo na fila e de atendimento [ARTIGO]
m_renda = 7.44 # renda média por hora do cidadão sergipano [IBGE]
F_visitas = 2000.0         # visitas/ano (frequência) [ARTIGO]

In [11]:
## Parâmetros: Custos - podem ser usados para orçamento, caso necessário 
CI = 1_800_000 + 1.5*207_365          # custo anual médio do centro - média de 5.14 profissionais por centro
S_e = {
    'Ortopedia': 87466.2,  # Exemplo em R$
    'Proctologia': 88902.84, # Exemplo em R$
    'Urologia': 91976.88   # Exemplo em R$
}

In [12]:
# Parâmetros: Outros

# Deslocamento máximo de um paciente para a especialidade e
DM_por_especialidade = {
    'Ortopedia': 180.0,
    'Proctologia': 180.0,
    'Urologia': 180.0
}

# Dicionários de Parâmetros
K_rotas_validas = set()  # Conjunto final de pares (i, j) válidos, K
W_eij = {}      # O termo final da FO: alpha_eij * CD_ij
DEMANDA_PONDERADA_J_E = {} # Denominador do R_j,e: Soma de d_i,e * w_ij para todos i que j cobre

# Iterar sobre as rotas e calcular o fator de decaimento (w_ij)
for _, row in rotas_df.iterrows():
    i = row['COD_ORIGEM']
    j = row['COD_DESTINO']
    dist_km = row['DISTANCIA_KM']
    
    # Ignorar rotas inválidas ou com distância zero (a menos que i=j)
    if dist_km <= 0 and i != j:
        continue
        
    # Fator de Decaimento (w_ij) = alpha_e do seu modelo original
    for e, DM_e in DM_por_especialidade.items():
        
        # Se a distância for aceitável para qualquer especialidade, o par (i, j) é um candidato de rota.
        if dist_km <= DM_e:
            K_rotas_validas.add((int(i), int(j)))
        
        # Cálculo do Decay (w_ij)
        if dist_km > DM_e:
            fator_decaimento = 0.0
        else:
            # Fórmula: 1 - (dist_km / DM_e) ou max(DM_e - dist_km, 0) / DM_e
            fator_decaimento = max(DM_e - dist_km, 0) / DM_e
            
        # Termo final de custo social a ser minimizado na FO
        W_eij[(e, i, j)] = fator_decaimento

# Tratamento dos Pares Diagonais (i = j) e preenchimento dos dicionários para i=j
for i in I:
    if i in J:
        # Garante que i=j seja uma rota válida e o decay factor seja 1.0
        if (i, i) not in K_rotas_validas:
            K_rotas_validas.add((int(i), int(i)))
        
        for e in DM_por_especialidade.keys():
            # Decay para i=j é sempre 1.0 (máxima acessibilidade)
            W_eij[(e, i, i)] = 1.0

# Pré-cálculo do Denominador (Weighted Demand) para R_j,e (Demanda Ponderada)
for e in E:
    for j in J:
        ponderacao_total = 0.0
        
        # Demanda de todos os i's que podem se ligar a j (N_j)
        for i in I:
            if (i, j) in K_rotas_validas and (e, i, j) in W_eij:
                
                # Demand d_i,e (em FTE)
                demanda_i_e = HE_dict.get(i, {}).get(e, 0.0) 
                
                # Decay w_ij
                decaimento_i_j = W_eij.get((e, i, j), 0.0)
                
                ponderacao_total += demanda_i_e * decaimento_i_j
                
        # Armazena o denominador (constante) do R_j,e
        DEMANDA_PONDERADA_J_E[(e, j)] = ponderacao_total

In [13]:
print(len(K_rotas_validas))

1673


In [14]:
# Parâmetro: se o município já possui Centro Médico
P_j = dict(zip(centros_df['CD_MUN'], (centros_df['Total'] > 0).astype(int)))

In [15]:
# Modelo (Gurobi)
m = gp.Model("Localizacao_Centros_Medicos")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2732058
Academic license 2732058 - for non-commercial use only - registered to na___@usp.br


In [16]:
# Variáveis (Gurobi)

# y[j]: variável de decisão binária para abertura/operação de um centro em j (antigo z[j])
y = m.addVars(J, vtype=GRB.BINARY, name="y_new")

# x_new[e,j]: variável de decisão inteira para o suprimento (FTEs novos) da especialidade e em j (antigo w[e,j])
x_new = m.addVars(E, J, vtype=GRB.INTEGER, name="x_new")

# --- NOVAS VARIÁVEIS AUXILIARES ---
# R[e, j]: Supply-to-demand Ratio R_j,e
R = m.addVars(E, J, vtype=GRB.CONTINUOUS, name="R")

# A[i, e]: Accessibility Score A_i,e
A = m.addVars(I, E, vtype=GRB.CONTINUOUS, name="A")

# Z: Equidade (acesso mínimo ponderado)
Z = m.addVar(vtype=GRB.CONTINUOUS, name="Z")
m.update()

In [17]:
# --- FUNÇÃO OBJETIVO (MAXIMIZAR) ---
# 1. Eficiência (Acesso Médio Ponderado)
eficiencia_termo = gp.quicksum(HE_dict[i][e] * A[i, e] for i in I for e in E if i in HE_dict and e in HE_dict[i]) / len(I)
# 2. Equidade (Acesso Mínimo)
equidade_termo = Z

m.setObjective(alpha * eficiencia_termo + (1 - alpha) * equidade_termo, GRB.MAXIMIZE)

In [18]:
# RESTRIÇÕES

# 1. R1: Alocação de Supply (FTEs) para a Razão R[e, j]
#    R[e, j] = Supply_j,e / Weighted_Demand_j,e
#    A restrição é reescrita como: R[e, j] * Weighted_Demand_j,e = Supply_j,e
#    O Supply_j,e é (KE_existente + x_new_ej)
for e in E:
    for j in J:
        # Demanda Ponderada (Denominador) pré-calculado
        demanda_pond = DEMANDA_PONDERADA_J_E.get((e, j), 0.0) 
        
        # Supply existente vs. novo supply (x_new)
        if P_j[j] == 1:
            capac_total = KE_dict.get(j, {}).get(e, 0.0)
        else:
            capac_total = x_new[e, j]

        # Regra de Segurança: Evitar divisão por zero se a demanda ponderada for zero
        if demanda_pond > 0.0001:
            # Como demanda_pond é uma constante (parâmetro), a restrição é linear
            m.addConstr(R[e, j] * demanda_pond == capac_total, name=f"R1_Ratio_{e}_{j}")
        else:
            # Se a demanda ponderada for zero, R[e, j] não deve contribuir para a acessibilidade
            m.addConstr(R[e, j] == 0.0, name=f"R1_Ratio_Zero_Demanda_{e}_{j}")

In [19]:
# 2. R2: Cálculo da Acessibilidade A[i, e]
#    A[i, e] = Sum_{j in M_i} w_ij * R[e, j]
for i in I:
    for e in E:
        acesso_score = gp.quicksum(W_eij[(e, i, j)] * R[e, j] # W_ij é o antigo alpha_CD_eij (decay)
                                    for j in J if (e, i, j) in W_eij)
        m.addConstr(A[i, e] == acesso_score, name=f"R2_Access_{i}_{e}")

In [20]:
# 3. R3: Restrição de Equidade (Linearização da parte MIN da FO)
#    Z <= Demanda_Ponderada Agregada * Score de Acessibilidade Agregada
# R6 Nova: Restrição de Equidade Mínima POR ESPECIALIDADE
for i in I:
    for e in E:
        
        # O acesso só é medido se houver demanda para aquela especialidade naquele município
        demanda_i_e = HE_dict.get(i, {}).get(e, 0.0)
        
        if demanda_i_e > 0.0001:
            # Acesso Ponderado (d_i,e * A_i,e)
            # Como A[i, e] é uma variável do Gurobi, esta é a forma de obter o produto na restrição.
            acesso_ponderado_i_e = demanda_i_e * A[i, e]
            
            # Z deve ser menor ou igual a CADA Acesso Ponderado (i, e)
            m.addConstr(Z <= acesso_ponderado_i_e, name=f"R3_Equidade_Min_Especialidade_{i}_{e}")

In [21]:
# 4. R4: Capacidade do Centro (Substitui R12)
# A soma dos especialistas alocados (x_new_e,j) deve estar entre [k_min, k_max] * y_j
# Se um centro é aberto (y_j=1), deve ter entre k_min e k_max FTEs.
for j in J:
    total_new_fte = gp.quicksum(x_new[e, j] for e in E)
    if P_j.get(j, 0) == 0:  # Aplicado apenas a novos centros
        # Se aberto, deve alocar pelo menos k_min FTEs (e permite que x_new[e, j] seja > 0)
        m.addConstr(total_new_fte <= k_max * y[j], name=f"R4a_CapMax_{j}")
        # Se aberto, deve alocar no máximo k_max FTEs
        m.addConstr(total_new_fte >= k_min * y[j], name=f"R4b_CapMin_{j}")
    else:  # Garante que centros existentes (P_j=1) permaneçam abertos (y_j=1) se o seu modelo precisar de expansão lá
        m.addConstr(y[j] >= P_j[j], name=f"R4c_Forcar_Existente_{j}")
        m.addConstr(total_new_fte >= 0, name=f"R4d_NonNeg_{j}")  # Allow zero or more new FTEs

In [22]:
# 5. R5: Quantidade de Facilidades (Substitui antigo R8)
# ou (Max_Centros_Medicos)
m.addConstr(gp.quicksum(y[j] for j in J if P_j[j] == 0) <= P_max, name="R5_Max_Novos_Centros")

# 6. R6: (Min_Centros_Medicos)
# Caso haja limites mínimos e máximos de Centros Médicos a serem abertos, adicionar essa restrição e alterar '==' para '<=' na R5.
# m.addConstr(gp.quicksum(y[j] for j in J) >= P_min, name="R6_Min_Centros_Abertos")

<gurobi.Constr *Awaiting Model Update*>

In [23]:
# Teste para ver a demanda (HE_ie) e disponibilidade (KE_ie) para cada especialidade e
for e in E:
    total_demand = sum(HE_dict[i].get(e, 0) for i in I)
    total_supply = sum(KE_dict[j].get(e, 0) for j in J if j in KE_dict)
    print(f"{e}: Demand = {total_demand:.2f}, Existing Supply = {total_supply:.2f}, Gap = {total_demand - total_supply:.2f}")

Ortopedia: Demand = 229.94, Existing Supply = 41.06, Gap = 188.88
Proctologia: Demand = 45.99, Existing Supply = 12.30, Gap = 33.69
Urologia: Demand = 68.98, Existing Supply = 14.08, Gap = 54.90


In [24]:
# 7. R7: Orçamento para Capacidade
m.addConstr(gp.quicksum(x_new[e, j] for e in E for j in J) <= Q_max, name="R7_Orcamento")

<gurobi.Constr *Awaiting Model Update*>

In [25]:
m.update()
print(f"\nModelo redefinido com: {m.numVars} variáveis, {m.numConstrs} restrições.")


Modelo redefinido com: 401 variáveis, 577 restrições.


In [26]:
# Configurações do ambiente para PERFORMANCE TUNING (a serem rodadas antes de m.optimize())
#m.setParam('TuneCriterion', 2) # 2 = Tempo (foco em velocidade). 1 = Gap (foco em qualidade)
## Define parâmetros para acelerar a solução
#m.setParam(GRB.Param.TimeLimit, 3600)  # Limite de 10 minutos
#m.setParam(GRB.Param.MIPGap, 0.001)    # Para quando o gap for <= 1%
#m.setParam(GRB.Param.Cuts, 2)       # Tenta melhorar o bound mais agressivamente
#m.setParam(GRB.Param.Heuristics, 0.5) # Aumenta a frequência de busca de soluções inteiras

# Execução da ferramenta de tuning
# O Gurobi irá testar combinações de parâmetros e buscar o melhor desempenho.
#m.tune() 

# Verifica se o tuning encontrou novos parâmetros
#if m.tuneResultCount > 0:
#    # Carrega o melhor conjunto de parâmetros
#    m.getTuneResult(0)
#    # Aplica o conjunto de parâmetros otimizado que o Gurobi encontrou
#    m.write('gurobi_tuned_parameters.prm')
#    m.read('gurobi_tuned_parameters.prm')
#    print("Parâmetros Otimizados Aplicados.")
#else:
#    print("O tuning não encontrou melhorias nos parâmetros.")

In [27]:
# Otimização
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Academic license 2732058 - for non-commercial use only - registered to na___@usp.br
Optimize a model with 577 rows, 401 columns and 5364 nonzeros
Model fingerprint: 0xee9765cb
Variable types: 301 continuous, 100 integer (25 binary)
Coefficient statistics:
  Matrix range     [9e-04, 2e+02]
  Objective range  [3e-04, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 1e+02]
Found heuristic solution: objective 0.4506599
Presolve removed 577 rows and 401 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 0.483993 0.45066 
No other solutions better than 0.483993

Optimal solut

In [28]:
# Extração e Análise de Resultados
if m.status == GRB.OPTIMAL or m.status == GRB.TIME_LIMIT or m.status == GRB.SOLUTION_LIMIT:
    
    print("\n--- RESULTADOS DO MODELO DE MAXIMIZAÇÃO DE ACESSO ---")
    
    # Valor da Função Objetivo
    print(f"Valor Ótimo da Função Objetivo (Eficiência + Equidade): {m.objVal:.4f}")
    
    # Resultado da Equidade (Z)
    Z_otimo = Z.x
    print(f"Acesso Mínimo (Equidade - Variável Z): {Z_otimo:.4f}")
    
    # Análise da Alocação de FTEs (Supply)
    alocacao_novos_ftes = {}
    total_novos_ftes = 0
    
    for e in E:
        for j in J:
            if x_new[e, j].x > 0.0001:
                if j not in alocacao_novos_ftes:
                    alocacao_novos_ftes[j] = {}
                alocacao_novos_ftes[j][e] = x_new[e, j].x
                total_novos_ftes += x_new[e, j].x

    print(f"\nTotal de Novos FTEs Contratados (Orçamento Q_max): {total_novos_ftes:.0f}")
    print("\nAlocação Detalhada de Novos Especialistas (FTEs):")
    for j, especialidades in alocacao_novos_ftes.items():
        soma_fte = sum(especialidades.values())
        detalhes = ', '.join([f"{e}: {v:.0f} FTEs" for e, v in especialidades.items()])
        print(f" - Centro {j}: Total {soma_fte:.0f} FTEs. ({detalhes})")
        
    # Análise do Score de Acessibilidade (A[i, e])
    acessibilidade_municipios = {}
    
    for i in I:
        acesso_total_i = 0.0
        acesso_por_especialidade = {}
        
        for e in E:
            acesso_e = A[i, e].x
            demanda_e = HE_dict.get(i, {}).get(e, 0.0)
            
            # Cálculo do Acesso Ponderado (d_i,e * A_i,e)
            acesso_ponderado_e = demanda_e * acesso_e
            acesso_total_i += acesso_ponderado_e
            acesso_por_especialidade[e] = acesso_e
            
        acessibilidade_municipios[i] = {
            'Acesso_Total_Ponderado': acesso_total_i,
            'Acesso_Min_Z_Equidade': Z_otimo,
            'Detalhes_Acesso_Especialidade': acesso_por_especialidade
        }
        
    # Encontrando o município com menor acesso (Equidade)
    pior_acesso_i = min(acessibilidade_municipios.items(), key=lambda item: item[1]['Acesso_Total_Ponderado'])[0]
    pior_acesso_valor = acessibilidade_municipios[pior_acesso_i]['Acesso_Total_Ponderado']
    
    print(f"\n--- ANÁLISE DE ACESSIBILIDADE ---")
    print(f"Município com o Pior Acesso Ponderado (Z): {pior_acesso_i} (Valor: {pior_acesso_valor:.4f})")
    
    # Exibir os 5 municípios com maior acesso (Eficiência)
    top_5_acesso = sorted(acessibilidade_municipios.items(), key=lambda item: item[1]['Acesso_Total_Ponderado'], reverse=True)[:5]
    print("\nTop 5 Municípios com Maior Acesso Ponderado (Eficiência):")
    for i, dados in top_5_acesso:
        print(f" - Município {i}: {dados['Acesso_Total_Ponderado']:.4f}")

else:
    print(f"O modelo não encontrou uma solução ótima. Status: {m.status}")

if m.status == GRB.INFEASIBLE:
      print("O modelo é inviável. Verifique as restrições de nível de serviço (F, Pq) ou os limites de capacidade.")


--- RESULTADOS DO MODELO DE MAXIMIZAÇÃO DE ACESSO ---
Valor Ótimo da Função Objetivo (Eficiência + Equidade): 0.4840
Acesso Mínimo (Equidade - Variável Z): 0.0056

Total de Novos FTEs Contratados (Orçamento Q_max): 5

Alocação Detalhada de Novos Especialistas (FTEs):
 - Centro 2806206: Total 5 FTEs. (Ortopedia: 5 FTEs)

--- ANÁLISE DE ACESSIBILIDADE ---
Município com o Pior Acesso Ponderado (Z): 2800100 (Valor: 0.0340)

Top 5 Municípios com Maior Acesso Ponderado (Eficiência):
 - Município 2800308: 24.0773
 - Município 2804805: 7.4096
 - Município 2806206: 5.6229
 - Município 2806701: 3.6590
 - Município 2802908: 3.3476


In [29]:
for e in E:
    avg_demand = sum(DEMANDA_PONDERADA_J_E.get((e, j), 0) for j in J) / len(J)
    print(f"Average weighted demand for {e}: {avg_demand:.2f}")
for e in E:
    total_demand = sum(HE_dict[i].get(e, 0) for i in I)
    total_supply = sum(KE_dict[j].get(e, 0) for j in J if j in KE_dict)
    print(f"{e}: Demand = {total_demand:.2f}, Existing Supply = {total_supply:.2f}, Gap = {total_demand - total_supply:.2f}")

Average weighted demand for Ortopedia: 96.51
Average weighted demand for Proctologia: 19.30
Average weighted demand for Urologia: 28.95
Ortopedia: Demand = 229.94, Existing Supply = 41.06, Gap = 188.88
Proctologia: Demand = 45.99, Existing Supply = 12.30, Gap = 33.69
Urologia: Demand = 68.98, Existing Supply = 14.08, Gap = 54.90


In [30]:
print(DEMANDA_PONDERADA_J_E)

{('Ortopedia', 2800308): 159.25629703333345, ('Ortopedia', 2800209): 2.0556, ('Ortopedia', 2800605): 154.03482004444447, ('Ortopedia', 2800670): 116.97845383333333, ('Ortopedia', 2801207): 21.00518891111111, ('Ortopedia', 2801306): 122.17972427222217, ('Ortopedia', 2801405): 93.19205143888888, ('Ortopedia', 2802106): 118.1182008277778, ('Ortopedia', 2802908): 144.21595779444436, ('Ortopedia', 2803005): 81.90162508333334, ('Ortopedia', 2803203): 146.5824986499999, ('Ortopedia', 2803500): 128.07000432222222, ('Ortopedia', 2803609): 156.0151371833333, ('Ortopedia', 2804508): 92.45816416666666, ('Ortopedia', 2804607): 127.75922590000002, ('Ortopedia', 2804805): 157.14610965555556, ('Ortopedia', 2805406): 37.51517950555556, ('Ortopedia', 2805505): 2.2217, ('Ortopedia', 2805604): 38.01109381666665, ('Ortopedia', 2805703): 87.36529415555555, ('Ortopedia', 2806206): 2.0825, ('Ortopedia', 2806701): 156.0632280333334, ('Ortopedia', 2807105): 101.35430532777774, ('Ortopedia', 2807402): 71.7292798

In [31]:
print(R)

{('Ortopedia', 2800308): <gurobi.Var R[Ortopedia,2800308] (value 0.15636430372852284)>, ('Ortopedia', 2800209): <gurobi.Var R[Ortopedia,2800209] (value 0.10279237205682039)>, ('Ortopedia', 2800605): <gurobi.Var R[Ortopedia,2800605] (value 0.005994099254529563)>, ('Ortopedia', 2800670): <gurobi.Var R[Ortopedia,2800670] (value 0.002068757040888853)>, ('Ortopedia', 2801207): <gurobi.Var R[Ortopedia,2801207] (value 0.01158285226710347)>, ('Ortopedia', 2801306): <gurobi.Var R[Ortopedia,2801306] (value 0.0023301738622823784)>, ('Ortopedia', 2801405): <gurobi.Var R[Ortopedia,2801405] (value 0.0018241899107830919)>, ('Ortopedia', 2802106): <gurobi.Var R[Ortopedia,2802106] (value 0.015989915074593953)>, ('Ortopedia', 2802908): <gurobi.Var R[Ortopedia,2802908] (value 0.009948274947803794)>, ('Ortopedia', 2803005): <gurobi.Var R[Ortopedia,2803005] (value 0.012005866782246518)>, ('Ortopedia', 2803203): <gurobi.Var R[Ortopedia,2803203] (value 0.0036791568227234634)>, ('Ortopedia', 2803500): <gurobi

In [32]:
print(x_new)

{('Ortopedia', 2800308): <gurobi.Var x_new[Ortopedia,2800308] (value -0.0)>, ('Ortopedia', 2800209): <gurobi.Var x_new[Ortopedia,2800209] (value -0.0)>, ('Ortopedia', 2800605): <gurobi.Var x_new[Ortopedia,2800605] (value -0.0)>, ('Ortopedia', 2800670): <gurobi.Var x_new[Ortopedia,2800670] (value -0.0)>, ('Ortopedia', 2801207): <gurobi.Var x_new[Ortopedia,2801207] (value -0.0)>, ('Ortopedia', 2801306): <gurobi.Var x_new[Ortopedia,2801306] (value -0.0)>, ('Ortopedia', 2801405): <gurobi.Var x_new[Ortopedia,2801405] (value -0.0)>, ('Ortopedia', 2802106): <gurobi.Var x_new[Ortopedia,2802106] (value -0.0)>, ('Ortopedia', 2802908): <gurobi.Var x_new[Ortopedia,2802908] (value -0.0)>, ('Ortopedia', 2803005): <gurobi.Var x_new[Ortopedia,2803005] (value -0.0)>, ('Ortopedia', 2803203): <gurobi.Var x_new[Ortopedia,2803203] (value -0.0)>, ('Ortopedia', 2803500): <gurobi.Var x_new[Ortopedia,2803500] (value -0.0)>, ('Ortopedia', 2803609): <gurobi.Var x_new[Ortopedia,2803609] (value -0.0)>, ('Ortopedia

In [33]:
for e in E:
        for j in J:
            if e == "Proctologia" and j in (2802106, 2805406, 2805703, 2802908):
                demanda_pond = DEMANDA_PONDERADA_J_E.get((e, j), 0.0)
                capac_total_val = KE_dict.get(j, {}).get(e, 0.0) if P_j[j] == 1 else x_new[e, j].x
                print(f"Município: {j}")
                print(f"Demanda ponderada: {demanda_pond}")
                print(f"x_e,j: {x_new[e,j].x}")
                print(f"Capacidade total: {capac_total_val}\n")

Município: 2802106
Demanda ponderada: 23.623640165555553
x_e,j: -0.0
Capacidade total: 0.6162

Município: 2802908
Demanda ponderada: 28.843191558888883
x_e,j: -0.0
Capacidade total: 0.2925

Município: 2805406
Demanda ponderada: 7.5030359011111125
x_e,j: -0.0
Capacidade total: 0.0387

Município: 2805703
Demanda ponderada: 17.473058831111107
x_e,j: -0.0
Capacidade total: 0.1725



In [34]:
print(P_j)

{2800100: 0, 2800209: 1, 2800308: 1, 2800407: 1, 2800506: 1, 2800605: 1, 2800670: 1, 2800704: 0, 2801009: 1, 2801108: 0, 2801207: 1, 2801306: 1, 2801405: 1, 2801504: 1, 2801603: 1, 2801702: 1, 2801900: 0, 2802007: 0, 2802106: 1, 2802205: 0, 2802304: 1, 2802403: 0, 2802502: 0, 2802601: 0, 2802700: 0, 2802809: 1, 2802908: 1, 2803005: 1, 2803104: 1, 2803203: 1, 2803302: 1, 2803401: 0, 2803500: 1, 2803609: 1, 2803708: 1, 2803807: 1, 2803906: 1, 2804003: 1, 2804102: 1, 2804201: 1, 2804300: 0, 2804409: 1, 2804458: 1, 2804508: 1, 2804607: 1, 2804706: 0, 2804805: 1, 2804904: 1, 2805000: 1, 2805109: 0, 2805208: 1, 2805307: 0, 2805406: 1, 2805505: 1, 2805604: 1, 2805703: 1, 2805802: 1, 2805901: 0, 2806008: 1, 2806107: 1, 2806206: 0, 2806305: 0, 2806404: 0, 2806503: 0, 2806602: 0, 2806701: 1, 2806800: 1, 2806909: 0, 2807006: 0, 2807105: 1, 2807204: 1, 2807303: 0, 2807402: 1, 2807501: 1, 2807600: 1}
